In [1]:
from torch_geometric.data import Data
from torch_geometric.loader import ClusterData, ClusterLoader
def load_data(self):
    config = self.config
    data_ = dataset_network.get_dataset(config)
    if config.dataset == 'icdm_test':
        features, edge_index, node_index = data_.data_tensor()
        return edge_index, features, node_index
    else:
        features, true_clusters, edge_index, node_index = data_.data_tensor()
        return edge_index, features, true_clusters, node_index
cluster_data_save_dir = '/home/chensiqi/lhtz/ACMin/data/icdm2023_session1_test/processed'
edge_index, features, node_index = self.load_data()
network_data = Data(x=features, edge_index=edge_index, node_index = node_index)
cluster_data = ClusterData(network_data, num_parts=10, recursive=True, save_dir=cluster_data_save_dir)
data_loader = ClusterLoader(cluster_data, batch_size=1, shuffle=True, num_workers=16)
for i, batch in enumerate(data_loader):
    print(batch.x.shape[0])


NameError: name 'network_data' is not defined

In [4]:
from sklearn.cluster import KMeans
import numpy as np
import os
# import cPickle as pickle
import pickle
import importlib
import torch
# from scipy.sparse.linalg.eigen.arpack import eigsh as largest_eigsh
# from scipy.sparse.linalg.eigen.arpack import eigs as largest_eigs
from scipy.linalg import qr
import time
from scipy.sparse import csc_matrix, csr_matrix
from numpy import linalg as LA
import random
from torch_geometric.data import Data
from torch_geometric.loader import ClusterData, ClusterLoader
from multiprocessing.pool import ThreadPool
import multiprocessing as mp
from tqdm import tqdm
import dataset_network

from config.networks import default as cfg

def solve_envs():
    mod = importlib.import_module(f'config.networks')
    cfg = getattr(mod, 'default')
    print(cfg)
    # update config
    for k, v in cfg.items():
            print(k, '\t', v)
    return cfg

def load_data(cfg):
    config = cfg
    data_ = dataset_network.get_dataset(config)
    if config.dataset == 'icdm_test':
        features, edge_index, node_index = data_.data_tensor()
        return edge_index, features, node_index
    else:
        features, true_clusters, edge_index, node_index = data_.data_tensor()
        return edge_index, features, true_clusters, node_index

if __name__ == '__main__':
    cfg = solve_envs()
    edge_index, features, node_index = load_data(cfg)
    network_data = Data(x=features, edge_index=edge_index, node_index = node_index)
    cluster_data_save_dir = 'cluster_data/'
    cluster_data = ClusterData(network_data, num_parts=10, recursive=False, save_dir=cluster_data_save_dir)
    data_loader = ClusterLoader(cluster_data, batch_size=1, shuffle=True, num_workers=16)
    del cluster_data
    del network_data
    num_batchs = len(data_loader)
    available_list = [i for i in range(features.shape[0])]
    print(f'origin_len: {len(available_list)}')
    node_dict = dict()
    for i, batch in enumerate(data_loader):
        def get_index(ith):
            nonzero_indices = torch.nonzero(torch.all(batch.x[ith] == features, dim=1))
            if nonzero_indices.shape[0]>1:
                for i_repeat in range(nonzero_indices.shape[0]):
                    ind_repeat = nonzero_indices[i_repeat].item()
                    if ind_repeat in available_list:
                        available_list.remove(ind_repeat)
                        return ind_repeat
            else:
                print(nonzero_indices.item())
                print(batch.node_index[ind])
                return nonzero_indices.item()       
        fs_list = [inde for inde in range(batch.x.shape[0])]
        fs_list = fs_list[:10]
        for index_fs in fs_list:
            get_index(index_fs)
    print(f'after_len: {len(available_list)}')
    file_path = "/home/chensiqi/lhtz/ACMin/data/icdm2023_session1_test/processed/node_list.pkl"
    with open(file_path, "wb") as file_:
        pickle.dump(node_dict, file_)


ACMin_cluster_params 	 {'alpha': 0.2, 'beta': 0.35, 't': 5, 'tmax': 200, 'ri': False, 'print_batch': 20}
batch_size 	 1
cluster_parts_num 	 10
cluster_result_path 	 results
data_total_path 	 data
dataset 	 icdm_test
mode 	 cluster
name 	 default
num_cluster 	 200


/home/chensiqi/lhtz/ACMin/dataset_network/base_network.py:117: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.x, dtype = torch.float), torch.tensor(edges_df), torch.tensor(self.nodes)


origin_len: 2365188


Traceback (most recent call last):
  File "/home/chensiqi/anaconda3/envs/py38/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/home/chensiqi/anaconda3/envs/py38/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
  File "/home/chensiqi/anaconda3/envs/py38/lib/python3.8/site-packages/torch/multiprocessing/reductions.py", line 369, in reduce_storage
    fd, size = storage._share_fd_cpu_()
RuntimeError: unable to write to file </torch_19389_1185661537_0>: No space left on device (28)
ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 Traceback (most recent call last):
  File "/home/chensiqi/anaconda3/envs/py38/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/home/ch

RuntimeError: DataLoader worker (pid(s) 19959) exited unexpectedly

Traceback (most recent call last):
  File "/home/chensiqi/anaconda3/envs/py38/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/home/chensiqi/anaconda3/envs/py38/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
  File "/home/chensiqi/anaconda3/envs/py38/lib/python3.8/site-packages/torch/multiprocessing/reductions.py", line 369, in reduce_storage
    fd, size = storage._share_fd_cpu_()
RuntimeError: unable to write to file </torch_19452_720880470_0>: No space left on device (28)
ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 Traceback (most recent call last):
  File "/home/chensiqi/anaconda3/envs/py38/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/home/chensiqi/anaconda3/envs/py38/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol